# Decorators

Decorators are higher order functions that accept functions and return another function that executes the original.

We define a function for which we want to validate/amend the parameters before calling the function.

In [2]:
def my_function(name: str, age: int):
    print("my_function: name =", name, "age =", age)

Let's start by defining a higher order function that returns a function that checks the parameters and calls the original function before returning this new parameter checking function.

In [3]:
def check_age(func):
    """checking age parameter higher order function - function that returns a function."""
    def do_checking(name, age):
        print("do_checking: validating input parameters")
        if age is None or age < 0:          # do the validation of the parameters
            age = 16
        return func(name, age)              # call the original function
    return do_checking

We can then create and call this revised value checking version of the same function

In [4]:
my_function("bob", -19)                                 # call function normally
print("Wrap my_function...")
my_function_with_checking = check_age(my_function)
my_function_with_checking("bob", -19)                   # call same function with checking wrapper function

my_function: name = bob age = -19
Wrap my_function...
do_checking: validating input parameters
my_function: name = bob age = 16


We can stack it again with another high order function

In [5]:
def capitalise_name(func):
    def do_capitilisation(name, age):
        print("do_capitilisation: ensure names are capitialised")
        name = name.capitalize()
        return func(name, age)
    return do_capitilisation

my_function_with_checking = check_age(my_function)
my_function_with_checking_and_capitilisation = capitalise_name(my_function_with_checking)
my_function_with_checking_and_capitilisation("bob", -19)

do_capitilisation: ensure names are capitialised
do_checking: validating input parameters
my_function: name = Bob age = 16


You can see this as a series of nested function calls with a function as parameter.

In [6]:
check_age(capitalise_name(my_function))("bob", -19)

do_checking: validating input parameters
do_capitilisation: ensure names are capitialised
my_function: name = Bob age = 16


These higher order functions are known as decorators and we can use the `@` symbol to simplify decoration of a function to do exactly what we have done above.

In [7]:
@check_age                                  # this decorator is called first
@capitalise_name
def my_function(name, age):
    print("my_function: name =", name, "age =", age)
    return age

print("my_function.__name__ =", my_function.__name__)       # not what we expected
ret_value = my_function("bob", -19)
print(ret_value)

my_function.__name__ = do_checking
do_checking: validating input parameters
do_capitilisation: ensure names are capitialised
my_function: name = Bob age = 16
16


We can use `args` and `kwargs` to make decorators suitable for different functions and parameters...

In [8]:
def my_general_capitalize_decorator(func):     # this general decorator will with any function
    def capitalise_func(*args, **kwargs):
        # iterate over all args and kwargs and capitalise any strings before calling real function
        args = tuple([x.capitalize() if isinstance(x, str) else x for x in args])
        kwargs = {k: v.capitalize() if isinstance(v, str) else v for k, v in kwargs.items()}
        func(*args, **kwargs)
    return capitalise_func

@my_general_capitalize_decorator
def my_function(name: str, age: int, surname: str):
    print("my_function: name =", name, surname, "age =", age)

@my_general_capitalize_decorator
def my_other_function(place: str, time: int):
    print("my_other_function: meet me in", place, "at", time, "hours")

my_function('bob', 34, surname='smith')
my_other_function('edinburgh', 1500)

my_function: name = Bob Smith age = 34
my_other_function: meet me in Edinburgh at 1500 hours


Will also work with class methods...

In [9]:
class SomeRandomClass:
    def __init__(self):
        pass

    @my_general_capitalize_decorator
    def my_method(self, name, age, surname):
        print("my_method: name =", name, surname, "age =", age)

my_instance = SomeRandomClass()
my_instance.my_method('bob', 34, surname='smith')

my_method: name = Bob Smith age = 34


or even a lambda...

In [10]:
my_general_capitalize_decorator(lambda x, y: print(x, y))('hello', 'bobby')

Hello Bobby


`wraps()` decorator from `functools` can be used to preserve original name and docstring

In [11]:
import functools

def my_decorator(func):
    @functools.wraps(func)              # note, you need to send func parameter in this case
    def do_decoration():
        print("do_decoration: Hello from decorator!")
        func()
    return do_decoration

@my_decorator
def my_function():
    """my_function doc string"""
    print("my_function: Hello from my_function!")

my_function()
print("my_function.__name__ =", my_function.__name__)
print("my_function.__doc__ =", my_function.__doc__)

do_decoration: Hello from decorator!
my_function: Hello from my_function!
my_function.__name__ = my_function
my_function.__doc__ = my_function doc string


Decorators can be very simple for debugging or registering...

In [12]:
def my_simple_decorator(func):
    print("my_simple_decorator:", func.__name__)    # this will be printed when function is decorated not..
    return func                                     # ..when the function is called

@my_simple_decorator                                # note that my_simple_decorator is applied here
def my_function():
    print("my_function: Hello from my_function")

my_function()

my_simple_decorator: my_function
my_function: Hello from my_function


We can pass parameters to a decorator using an extra function wrapper. For example the decorator `functool.wraps()` takes a function object as a parameter.

In [13]:
def my_param_decorator(prefix, loops):
    print("my_param_decorator")
    def my_parameterised_decorator(func):
        print("my_parameterised_decorator")
        def do_decoration(name):
            print("do_decoration")
            for i in range(loops):
                func(prefix + name)
        return do_decoration
    return my_parameterised_decorator

@my_param_decorator("Name = ", 2)         # my_param_decorator and my_parameterised_decorator called here
def my_function(name):
    print("my_function:", name)

print("Call my_function")
my_function("Bob")                        # do_decoration is done here

my_param_decorator
my_parameterised_decorator
Call my_function
do_decoration
my_function: Name = Bob
my_function: Name = Bob


It is possible to make the decorator parameters optional with default values, as well as being able to add state to the decorator. Thanks to https://realpython.com/primer-on-python-decorators/.

In [14]:
def my_param_decorator(_func=None, *, number_function_loops=1):
    """The asterisk means that all parameters afterwards are keyword only"""
    def my_parameterised_decorator(func):
        def do_decoration(*args, **kwargs):
            do_decoration.number_decorations += 1       # decorator state update
            for i in range(number_function_loops):
                func(*args, **kwargs)
        do_decoration.number_decorations = 0            # we can add attributes as usual for state
        return do_decoration

    if _func is None:
        print("my_param_decorator: _func is None so parameters were specified")
        print("my_param_decorator: number_function_loops =", number_function_loops)
        return my_parameterised_decorator
    else:
        print("my_param_decorator: _func is", _func)
        print("my_param_decorator: no parameters specified, calling my_parameterised_decorator")
        _decorator_func = my_parameterised_decorator(_func)
        print("my_param_decorator: called my_parameterised_decorator to get decorator function")
        return _decorator_func

Calling function with non-parameterised decorator...

In [15]:
@my_param_decorator         # so this is effectively my_param_decorator(my_function) so _func = my_function
def my_function():
    print("my_function")

my_function()
print("Number of decorations:", my_function.number_decorations)

my_param_decorator: _func is <function my_function at 0x0000020857B2F6D0>
my_param_decorator: no parameters specified, calling my_parameterised_decorator
my_param_decorator: called my_parameterised_decorator to get decorator function
my_function
Number of decorations: 1


Calling function with parameterised decorator...

In [16]:
@my_param_decorator(number_function_loops=2)  # have keyword parameters so _func = None so this is effectively...
def my_function():                            # ...my_param_decorator(number_function_loops=2)(my_function)
    print("my_function")

my_function()
print("number of decorations:", my_function.number_decorations)
my_function()
print("number of decorations:", my_function.number_decorations)

my_param_decorator: _func is None so parameters were specified
my_param_decorator: number_function_loops = 2
my_function
my_function
number of decorations: 1
my_function
my_function
number of decorations: 2
